In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='5e647a80-7f51-4fde-8506-7414478c39fa'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T14:37:50.249946849Z', 'done': True, 'done_reason': 'stop', 'total_duration': 405765653681, 'load_duration': 53673753192, 'prompt_eval_count': 1066, 'prompt_eval_duration': 99661620184, 'eval_count': 1092, 'eval_duration': 250918449305, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9535-c3f7-72b1-be02-62f210569e8e-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'curr': 'USD', 'departureDate': '31/03/2026', 'departureDateFlexRange': 0, 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1}, 'returnDate': '31/03/2026', 'returnDateFlexRange': 0, 'sort': 'price'}, 'id': '976c52

In [7]:
print(response["messages"][-1].content)

I’m sorry, but there are no direct flights from San Francisco to Tokyo on 31 March 2026.  
You might consider:

- **Flights with one layover** (e.g., via Los Angeles, Seattle, or a major Asian hub).  
- **Adjusting the travel date** by a day or two to catch a direct service.  
- **Checking alternative airports** such as Oakland (OAK) or San Jose (SJC) for better options.

**Fun fact:** Tokyo’s Shibuya Crossing is the world’s busiest pedestrian scramble, with up to 3,000 people crossing at once during peak hours! Have a great trip!


In [8]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Can you then check for the next two days")]},
    config
    )

In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='5e647a80-7f51-4fde-8506-7414478c39fa'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-25T14:37:50.249946849Z', 'done': True, 'done_reason': 'stop', 'total_duration': 405765653681, 'load_duration': 53673753192, 'prompt_eval_count': 1066, 'prompt_eval_duration': 99661620184, 'eval_count': 1092, 'eval_duration': 250918449305, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9535-c3f7-72b1-be02-62f210569e8e-0', tool_calls=[{'name': 'search-flight', 'args': {'cabinClass': 'M', 'curr': 'USD', 'departureDate': '31/03/2026', 'departureDateFlexRange': 0, 'flyFrom': 'San Francisco', 'flyTo': 'Tokyo', 'locale': 'en', 'passengers': {'adults': 1}, 'returnDate': '31/03/2026', 'returnDateFlexRange': 0, 'sort': 'price'}, 'id': '976c52

In [10]:
print(response["messages"][-1].content)

It looks like there are no direct flights from **San Francisco (SFO)** to **Tokyo (NRT or HND)** on 31 March 2026 or the following two days.  
The most common reason for this is that the airlines only operate direct services on specific days of the week, and those days don’t line up with the dates you requested.

**What you can do next:**

| Option | What it means | Why it helps |
|--------|---------------|--------------|
| **Try a one‑stop flight** | Flights that connect in a major hub (e.g., Los Angeles, Seattle, or a Japanese hub like Osaka) | These are usually available every day and can be cheaper |
| **Shift your travel date by a day or two** | Pick a date that falls on a day when a direct flight is scheduled | Direct flights often run on specific weekdays |
| **Use a different departure airport** | Check Oakland (OAK) or San Jose (SJC) | Some airlines schedule direct services from these airports on different days |

If you’d like, I can run a search for the next available direct

In [ ]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Check between March 27 and 29.")]},
    config
    )

In [ ]:
from pprint import pprint

pprint(response)

In [ ]:
print(response["messages"][-1].content)